# V11

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from scipy.stats import f

pd.set_option('Display.max_columns', 100)

## 1. 담합이 없을 경우 (회귀분석자료.xlsx)

In [2]:
dfReg = pd.read_excel("./교수님 전달 파일/회귀분석자료 수정.xlsx")

In [3]:
dfReg.loc[dfReg['연도'].str.contains("-"), '연도'] = dfReg.loc[dfReg['연도'].str.contains("-"), '연도'].apply(lambda x : x.split('-')[0])
dfReg.loc[dfReg['연도'].str.contains("B"), '연도'] = dfReg.loc[dfReg['연도'].str.contains("B"), '연도'].apply(lambda x: x.split('B5')[1][0:4])

In [4]:
dfReg['연도'].unique()

array(['2011', '2012', '2013', '2014', '2016', '2017', '2018', '2019',
       '2020'], dtype=object)

### Year Dummy 11 ~ 20년도 각각

In [6]:
df_temp1 = dfReg[['연도','업체수','업체수TH','budget','tenderRatio','techScore','techGap','낙찰여부','largeBusiness']]
df_temp1 = df_temp1.copy()
df_temp1[['d2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019']] = 0


In [7]:
df_temp1.loc[df_temp1['연도'] == "2011", 'd2011'] = 1
df_temp1.loc[df_temp1['연도'] == "2012", 'd2012'] = 1
df_temp1.loc[df_temp1['연도'] == "2013", 'd2013'] = 1
df_temp1.loc[df_temp1['연도'] == "2014", 'd2014'] = 1
df_temp1.loc[df_temp1['연도'] == "2015", 'd2014'] = 1 ##### 
df_temp1.loc[df_temp1['연도'] == "2016", 'd2016'] = 1
df_temp1.loc[df_temp1['연도'] == "2017", 'd2017'] = 1
df_temp1.loc[df_temp1['연도'] == "2018", 'd2018'] = 1
df_temp1.loc[df_temp1['연도'] == "2019", 'd2019'] = 1

df_temp1

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019
0,2011,3,3,1514040000,77.105549,94.110000,0.000,NaN,1,1,0,0,0,0,0,0,0
1,2011,3,3,1514040000,79.018773,93.060000,0.735,낙찰,0,1,0,0,0,0,0,0,0
2,2011,3,3,1514040000,94.219732,90.980000,2.191,NaN,1,1,0,0,0,0,0,0,0
3,2011,7,7,1947383000,75.039882,98.815714,0.000,낙찰,0,1,0,0,0,0,0,0,0
4,2011,7,7,1947383000,75.197976,96.865714,1.365,NaN,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2020,9,9,1321100000,73.634457,99.360000,0.448,낙찰,1,0,0,0,0,0,0,0,0
368,2020,4,4,1540000000,73.686583,100.000000,0.000,낙찰,0,0,0,0,0,0,0,0,0
369,2020,4,4,1540000000,73.709675,96.180000,2.674,NaN,0,0,0,0,0,0,0,0,0
370,2020,4,4,1540000000,77.050700,95.720000,2.996,NaN,0,0,0,0,0,0,0,0,0


### Year dummy 13년도 이전 & 이후

In [8]:
df_temp2 = df_temp1.copy()
df_temp2['db2014'] = 0
#df_temp2['da2013'] = 0
df_temp2.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014
0,2011,3,3,1514040000,77.105549,94.11,0.000,NaN,1,1,0,0,0,0,0,0,0,0
1,2011,3,3,1514040000,79.018773,93.06,0.735,낙찰,0,1,0,0,0,0,0,0,0,0


In [9]:
# 2014년도 이전 및 2013년도 이후 구별해서 더미 생성
year_d1 = ['2011', '2012', '2013','2014']
#year_d2 = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
df_temp2.loc[df_temp2['연도'].isin(year_d1), 'db2014'] = 1
#df_temp2.loc[df_temp2['입찰공고번호'].isin(year_d2), 'da2013'] = 1

In [10]:
df_temp2.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014
0,2011,3,3,1514040000,77.105549,94.11,0.000,NaN,1,1,0,0,0,0,0,0,0,1
1,2011,3,3,1514040000,79.018773,93.06,0.735,낙찰,0,1,0,0,0,0,0,0,0,1


### 평균 대비 설계 점수 & 평균 대비 설계 점수 비율 변수 생성

In [11]:
df_temp3 = df_temp2.copy()
df_temp3['mtScore'] = df_temp3['techScore'] - np.mean(df_temp3['techScore']) # mean = 67.2576
df_temp3['mtRatio'] = df_temp3['mtScore']/np.mean(df_temp3['techScore'])
df_temp3.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio
0,2011,3,3,1514040000,77.105549,94.11,0.000,NaN,1,1,0,0,0,0,0,0,0,1,-1.97235,-0.020528
1,2011,3,3,1514040000,79.018773,93.06,0.735,낙찰,0,1,0,0,0,0,0,0,0,1,-3.02235,-0.031456


### TechGap Dummy (양수이면 1 아니면 0)

In [12]:
df_temp4 = df_temp3.copy()
df_temp4['dtechGap'] = 0
df_temp4.loc[df_temp4['techGap'] > 0, 'dtechGap'] = 1
df_temp4.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap
0,2011,3,3,1514040000,77.105549,94.11,0.000,NaN,1,1,0,0,0,0,0,0,0,1,-1.97235,-0.020528,0
1,2011,3,3,1514040000,79.018773,93.06,0.735,낙찰,0,1,0,0,0,0,0,0,0,1,-3.02235,-0.031456,1


### 낙찰이 되었을 경우

In [13]:
df_temp5 = df_temp4.copy()
df_temp5 = df_temp5.loc[df_temp4['낙찰여부'] == '낙찰']
df_temp5.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap
1,2011,3,3,1514040000,79.018773,93.060000,0.735,낙찰,0,1,0,0,0,0,0,0,0,1,-3.022350,-0.031456,1
3,2011,7,7,1947383000,75.039882,98.815714,0.000,낙찰,0,1,0,0,0,0,0,0,0,1,2.733364,0.028448,0


In [32]:
df_temp5 = df_temp5.copy()
df_temp5['lnCash'] = np.log(df_temp5['budget'])
df_temp5.head(3)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap,lnCash
1,2011,3,3,1514040000,79.019,93.060,0.735,낙찰,0,1,0,0,0,0,0,0,0,1,-3.022,-0.031,1,21.138
3,2011,7,7,1947383000,75.040,98.816,0.000,낙찰,0,1,0,0,0,0,0,0,0,1,2.733,0.028,0,21.390
10,2011,2,2,1429000000,76.853,100.000,0.000,낙찰,1,1,0,0,0,0,0,0,0,1,3.918,0.041,0,21.080


### FITTED VALUE

In [18]:
fit_final = pd.read_csv("./df_fit.csv")

In [19]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
fit_final

,tenderRatio,lnCash,techScore,techGap,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap
0,87.130,23.323,94.702,2.130,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-0.183,-0.002,1
1,87.950,23.336,94.702,2.642,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-0.183,-0.002,1
2,86.310,21.892,93.852,1.020,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-1.033,-0.011,1
3,86.440,22.519,93.495,0.953,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-1.389,-0.015,1
4,85.160,23.162,94.732,2.378,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-0.153,-0.002,1
5,87.500,22.691,92.043,-0.548,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-2.841,-0.030,0
6,87.140,22.604,93.838,1.305,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-1.046,-0.011,1
7,84.830,23.352,97.952,5.546,2.000,2.000,0,0,1,0,0,0,0,0,0,1,3.067,0.032,1
8,87.080,23.495,97.908,6.835,2.000,2.000,0,0,1,0,0,0,0,0,0,1,3.024,0.032,1
9,85.240,22.546,94.637,3.396,2.000,2.000,0,0,1,0,0,0,0,0,0,1,-0.248,-0.003,1


### 담합이 없을 경우 (374 & 76개) ols 결과

In [ ]:
# 낙찰 X : 기존 점수 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ techScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ techScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ techScore + techGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ techScore + 업체수 + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-0.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-0.csv', index = False)

# 낙찰 X : 기존 점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-0(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-0(TH).csv', index = False)


In [ ]:
# 낙찰 X : 평균대비설계점수 기준 & 업체수 wiothout Threshold

results1 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ mtScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ mtScore + 업체수 + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-1.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-1.csv', index = False)

# 낙찰 X : 평균대비설계점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-1(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-1(TH).csv', index = False)


In [ ]:
# 낙찰 X : 평균대비설계점수 기준 & 업체수 wiothout Threshold

results1 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ mtRatio + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ mtRatio + 업체수 + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-2.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-2.csv', index = False)


# 낙찰 X : 평균대비설계점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results2 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()
results3 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp4).fit()

results4 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results5 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()
results6 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + db2014', data=df_temp4).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v1-2(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v1-2(TH).csv', index = False)


### 낙찰 O

In [ ]:
# 낙찰 O : 기존 점수 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ techScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ techScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ techScore + techGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ techScore + 업체수 + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-0.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-0.csv', index = False)

# 낙찰 X : 기존 점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-0(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-0(TH).csv', index = False)


In [ ]:
# 낙찰 O : 평균대비설계점수 점수 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ mtScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ mtScore + 업체수 + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-1.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-1.csv', index = False)


# 낙찰 X : 평균대비설계점수 점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-1(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-1(TH).csv', index = False)


In [ ]:
# 낙찰 O : 평균대비설계점수 비율 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ mtRatio + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수 + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ mtRatio + 업체수 + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-2.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-2.csv', index = False)

# 낙찰 X : 평균대비설계점수 비율 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results2 = smf.ols('tenderRatio ~ mtRatio+ dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()
results3 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019', data=df_temp5).fit()

results4 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results5 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()
results6 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + db2014', data=df_temp5).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V11_new/regression_v2-2(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final['tenderRatio'], results1.predict(fit_final), results2.predict(fit_final), results3.predict(fit_final),
                    results4.predict(fit_final), results5.predict(fit_final), results6.predict(fit_final)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v2-2(TH).csv', index = False)

## 2. 담합이 있을 경우 (참고자료 1.xlsx)

In [33]:
dfCol = pd.read_excel("./교수님 전달 파일/담합.xlsx", sheet_name='Sheet1', header=0)

In [34]:
dfCol.head(2)

,연도수정,연도,입찰 참가자,기초금액,"투찰금액 (천원, VAT 포함)",투찰률 (%),기술점수,기술점수 original,가격점수,총점,차이,비고,담합,업체수,업체수TH
0,NaN,11,수자원기술(100),13465052220.819,11732100.000,87.130,94.702,56.821,39.130,95.951,2.130,낙,1.000,2.000,2.000
1,NaN,11,환경(100),NaN,11784900.000,87.520,NaN,54.691,39.520,94.211,NaN,-,NaN,NaN,NaN


In [35]:
df_ctemp1 = dfCol[['연도', '기초금액','투찰률 (%)', '기술점수', '차이', '비고','담합','업체수','업체수TH']]
df_ctemp1.columns = ['Year', '기초금액','tenderRatio', 'techScore', 'techGap', '비고', 'collude','업체수','업체수TH']
df_ctemp2 = df_ctemp1.loc[df_ctemp1['비고'] == '낙']
df_ctemp2.tail(8)

,Year,기초금액,tenderRatio,techScore,techGap,비고,collude,업체수,업체수TH
52,15,8589933673.469,78.400,94.732,2.222,낙,1.000,2.000,2.000
54,16,10700334183.673,78.400,94.913,3.635,낙,1.000,2.000,2.000
55,15,11222392852.518,77.230,94.295,0.684,낙,0.000,2.000,2.000
58,16,37609553152.684,76.760,95.047,2.510,낙,0.000,2.000,2.000
60,16,34491902775.968,76.730,95.003,2.410,낙,0.000,2.000,2.000
62,16,23112492985.717,77.821,94.083,1.198,낙,0.000,2.000,2.000
64,16,27323585974.642,77.602,94.837,2.148,낙,0.000,2.000,2.000
66,16,13481235080.951,77.751,94.548,1.156,낙,0.000,2.000,2.000


In [36]:
df_ctemp2['Year'].unique()

array([11, 12, 13, 15, 16], dtype=int64)

In [37]:
df_ctemp3 = df_ctemp2.copy()
df_ctemp3['largeBusiness'] = 0
df_ctemp3[['d2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019']] = 0


df_ctemp3.loc[df_ctemp3['Year'] == 11, 'd2011'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 12, 'd2012'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 13, 'd2013'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 14, 'd2014'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 15, 'd2014'] = 1 #####
df_ctemp3.loc[df_ctemp3['Year'] == 16, 'd2016'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 17, 'd2017'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 18, 'd2018'] = 1
df_ctemp3.loc[df_ctemp3['Year'] == 19, 'd2019'] = 1

In [38]:
df_ctemp4 = df_ctemp3.copy()
df_ctemp4['db2014'] = 0
#df_ctemp4['da2013'] = 0
df_ctemp4.head(2)

,Year,기초금액,tenderRatio,techScore,techGap,비고,collude,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014
0,11,13465052220.819,87.130,94.702,2.130,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,0
2,11,13630358158.044,87.950,94.702,2.642,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,0


In [39]:
# 2014년도 이전 및 2013년도 이후 구별해서 더미 생성
year_d1 = [11,12,13,14]
#year_d2 = [14,15,16]
df_ctemp4.loc[df_ctemp4['Year'].isin(year_d1), 'db2014'] = 1
#df_ctemp4.loc[df_ctemp4['Year'].isin(year_d2), 'da2013'] = 1

In [40]:
df_ctemp4['lnCash'] = np.log(df_ctemp4['기초금액'])
df_ctemp4.head(3)

,Year,기초금액,tenderRatio,techScore,techGap,비고,collude,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,lnCash
0,11,13465052220.819,87.130,94.702,2.130,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,23.323
2,11,13630358158.044,87.950,94.702,2.642,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,23.336
4,11,3218051210.752,86.310,93.852,1.020,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,21.892


In [41]:
# 평균대비설계점수 & 비율
df_ctemp5 = df_ctemp4.copy()
df_ctemp5['mtScore'] = df_ctemp5['techScore'] - np.mean(df_ctemp5.loc[df_ctemp5['collude'] == 1,'techScore'])
df_ctemp5['mtRatio'] = df_ctemp5['mtScore']/np.mean(df_ctemp5.loc[df_ctemp5['collude'] == 1,'techScore'])

# TechGap Dummy
df_ctemp5['dtechGap'] = 0
df_ctemp5.loc[df_ctemp5['techGap'] > 0, 'dtechGap'] = 1

df_ctemp5.head(10)

,Year,기초금액,tenderRatio,techScore,techGap,비고,collude,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,lnCash,mtScore,mtRatio,dtechGap
0,11,13465052220.819,87.130,94.702,2.130,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,23.323,-0.183,-0.002,1
2,11,13630358158.044,87.950,94.702,2.642,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,23.336,-0.183,-0.002,1
4,11,3218051210.752,86.310,93.852,1.020,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,21.892,-1.033,-0.011,1
6,11,6026145303.100,86.440,93.495,0.953,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,22.519,-1.389,-0.015,1
8,11,11462071395.021,85.160,94.732,2.378,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,23.162,-0.153,-0.002,1
10,11,7154171428.571,87.500,92.043,-0.548,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,22.691,-2.841,-0.030,0
12,11,6558228138.627,87.140,93.838,1.305,낙,1.000,2.000,2.000,0,1,0,0,0,0,0,0,0,1,22.604,-1.046,-0.011,1
14,12,13860662501.474,84.830,97.952,5.546,낙,1.000,2.000,2.000,0,0,1,0,0,0,0,0,0,1,23.352,3.067,0.032,1
16,12,15993236104.731,87.080,97.908,6.835,낙,1.000,2.000,2.000,0,0,1,0,0,0,0,0,0,1,23.495,3.024,0.032,1
18,12,6191177850.774,85.240,94.637,3.396,낙,1.000,2.000,2.000,0,0,1,0,0,0,0,0,0,1,22.546,-0.248,-0.003,1


In [42]:
# 기존 df collude 항목 만들어주자
df_temp6 = df_temp5.copy()
df_temp6['collude'] = 0
# df_temp6['d2015'] = 0
df_temp6.head(2)

,연도,업체수,업체수TH,budget,tenderRatio,techScore,techGap,낙찰여부,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap,lnCash,collude
1,2011,3,3,1514040000,79.019,93.060,0.735,낙찰,0,1,0,0,0,0,0,0,0,1,-3.022,-0.031,1,21.138,0
3,2011,7,7,1947383000,75.040,98.816,0.000,낙찰,0,1,0,0,0,0,0,0,0,1,2.733,0.028,0,21.390,0


In [45]:
df_ctemp_final = df_ctemp5[['tenderRatio', 'lnCash', 'techScore', 'techGap', '업체수', '업체수TH', 'largeBusiness', 'd2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019',
                           'db2014', 'mtScore', 'mtRatio', 'dtechGap','collude']]

df_temp_final = df_temp6[['tenderRatio', 'lnCash', 'techScore', 'techGap', '업체수', '업체수TH', 'largeBusiness', 'd2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019', 
                           'db2014', 'mtScore', 'mtRatio', 'dtechGap','collude']]

df_final = df_temp_final.append(df_ctemp_final)
df_final.reset_index(drop = True, inplace = True)
df_final.isna().sum() # 0
df_final.head(3)

,tenderRatio,lnCash,techScore,techGap,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap,collude
0,79.019,21.138,93.060,0.735,3.000,3.000,0,1,0,0,0,0,0,0,0,1,-3.022,-0.031,1,0.000
1,75.040,21.390,98.816,0.000,7.000,7.000,0,1,0,0,0,0,0,0,0,1,2.733,0.028,0,0.000
2,76.853,21.080,100.000,0.000,2.000,2.000,1,1,0,0,0,0,0,0,0,1,3.918,0.041,0,0.000


In [46]:
fit = pd.read_excel("./교수님 전달 파일/담합.xlsx", sheet_name='Sheet1', header=0)
fit_temp1 = fit[['연도', '기초금액', '투찰률 (%)', '기술점수', '차이', '비고','담합','업체수','업체수TH']]
fit_temp1.columns = columns = ['Year', '기초금액', 'tenderRatio', 'techScore', 'techGap', '비고', 'collude','업체수','업체수TH']
fit_temp2 = fit_temp1.loc[(fit_temp1['비고'] == '낙') & (fit_temp1['collude'] == 1)]
fit_temp2['largeBusiness'] = 0


fit_temp2[['d2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019']] = 0

fit_temp2.loc[fit_temp2['Year'] == 11, 'd2011'] = 1
fit_temp2.loc[fit_temp2['Year'] == 12, 'd2012'] = 1
fit_temp2.loc[fit_temp2['Year'] == 13, 'd2013'] = 1
fit_temp2.loc[fit_temp2['Year'] == 14, 'd2014'] = 1
fit_temp2.loc[fit_temp2['Year'] == 15, 'd2014'] = 1 ############ 
fit_temp2.loc[fit_temp2['Year'] == 16, 'd2016'] = 1
fit_temp2.loc[fit_temp2['Year'] == 17, 'd2017'] = 1
fit_temp2.loc[fit_temp2['Year'] == 18, 'd2018'] = 1
fit_temp2.loc[fit_temp2['Year'] == 19, 'd2019'] = 1

fit_temp2['db2014'] = 0
year_d1 = [11,12,13,14]
fit_temp2.loc[fit_temp2['Year'].isin(year_d1), 'db2014'] = 1
fit_temp2['mtScore'] = fit_temp2['techScore'] - np.mean(fit_temp2.loc[fit_temp2['collude'] == 1, 'techScore'])
fit_temp2['mtRatio'] = fit_temp2['mtScore']/np.mean(fit_temp2.loc[fit_temp2['collude'] == 1, 'techScore'])
fit_temp2['dtechGap'] = 0
fit_temp2.loc[fit_temp2['techGap'] > 0, 'dtechGap'] = 1
fit_temp2.head(2)

fit_temp2['lnCash'] = np.log(fit_temp2['기초금액'])

fit_final = fit_temp2[['tenderRatio','lnCash','techScore', 'techGap', '업체수', '업체수TH', 'largeBusiness', 'd2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019',
                           'db2014', 'mtScore', 'mtRatio', 'dtechGap']]

fit_final.head(4)

c:\Users\user\anaconda3\envs\energy\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\user\anaconda3\envs\energy\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\user\anaconda3\envs\energy\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,tenderRatio,lnCash,techScore,techGap,업체수,업체수TH,largeBusiness,d2011,d2012,d2013,d2014,d2016,d2017,d2018,d2019,db2014,mtScore,mtRatio,dtechGap
0,87.130,23.323,94.702,2.130,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-0.183,-0.002,1
2,87.950,23.336,94.702,2.642,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-0.183,-0.002,1
4,86.310,21.892,93.852,1.020,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-1.033,-0.011,1
6,86.440,22.519,93.495,0.953,2.000,2.000,0,1,0,0,0,0,0,0,0,1,-1.389,-0.015,1


In [47]:
fit_final_col = fit_temp2[['tenderRatio','lnCash','techScore', 'techGap', '업체수', '업체수TH', 'largeBusiness', 'd2011', 'd2012', 'd2013', 'd2014', 'd2016', 'd2017', 'd2018', 'd2019',
                           'db2014', 'mtScore', 'mtRatio', 'dtechGap', 'collude']]

### V3 techScore

In [48]:
# 낙찰 O : 기존 점수 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ lnCash + techScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results2 = smf.ols('tenderRatio ~ lnCash + techScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results3 = smf.ols('tenderRatio ~ lnCash + techScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

results4 = smf.ols('tenderRatio ~ lnCash + techScore + techGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results5 = smf.ols('tenderRatio ~ lnCash + techScore + dtechGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results6 = smf.ols('tenderRatio ~ lnCash + techScore + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V12/regression_v3-0.html', 'w').write(stargazer_tab.render_html())  # for latex


6704

In [49]:

fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
                    results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V12/fitted_v3-0.csv', index = False)


In [32]:


# 낙찰 O : 기존 점수 기준 & 업체수 with Threshold

results1 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results2 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results3 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

results4 = smf.ols('tenderRatio ~ techScore + techGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
results5 = smf.ols('tenderRatio ~ techScore + dtechGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
results6 = smf.ols('tenderRatio ~ techScore + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V12/regression_v3-0(TH).html', 'w').write(stargazer_tab.render_html())  # for latex



6382

In [33]:
fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
                    results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V11_new/fitted_v3-0(TH).csv', index = False)

In [50]:
# 낙찰 O : 평균대비설계점수 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ lnCash + mtScore + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results2 = smf.ols('tenderRatio ~ lnCash + mtScore + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results3 = smf.ols('tenderRatio ~ lnCash + mtScore + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

results4 = smf.ols('tenderRatio ~ lnCash + mtScore + techGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results5 = smf.ols('tenderRatio ~ lnCash + mtScore + dtechGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results6 = smf.ols('tenderRatio ~ lnCash + mtScore + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V12/regression_v3-1.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
                    results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V12/fitted_v3-1.csv', index = False)

# # 낙찰 O : 평균대비설계점수 기준  & 업체수 with Threshold

# results1 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
# results2 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
# results3 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

# results4 = smf.ols('tenderRatio ~ mtScore + techGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
# results5 = smf.ols('tenderRatio ~ mtScore + dtechGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
# results6 = smf.ols('tenderRatio ~ mtScore + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()

# stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
# open('./V11_new/regression_v3-1(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

# fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
#                     results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
# fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
# fitted_final = fitted.copy()
# fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
# fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
# fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
# fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
# fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
# fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
# fitted_final.to_csv('./V11_new/fitted_v3-1(TH).csv', index = False)

In [52]:
# 낙찰 O : 평균대비설계점수 비율 기준 & 업체수 without Threshold

results1 = smf.ols('tenderRatio ~ lnCash + mtRatio + techGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results2 = smf.ols('tenderRatio ~ lnCash + mtRatio + dtechGap + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
results3 = smf.ols('tenderRatio ~ lnCash + mtRatio + 업체수 + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

results4 = smf.ols('tenderRatio ~ lnCash + mtRatio + techGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results5 = smf.ols('tenderRatio ~ lnCash + mtRatio + dtechGap + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()
results6 = smf.ols('tenderRatio ~ lnCash +mtRatio + 업체수 + largeBusiness + db2014 + collude', data=df_final).fit()

stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
open('./V12/regression_v3-2.html', 'w').write(stargazer_tab.render_html())  # for latex

fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
                    results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
fitted_final = fitted.copy()
fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
fitted_final.to_csv('./V12/fitted_v3-2.csv', index = False)

# # 낙찰 O : 평균대비설계점수 비율 기준 & 업체수 with Threshold

# results1 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014  + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
# results2 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()
# results3 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + d2011 + d2012 + d2013 + d2014 + d2016 + d2017 + d2018 + d2019 + collude', data=df_final).fit()

# results4 = smf.ols('tenderRatio ~ mtRatio + techGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
# results5 = smf.ols('tenderRatio ~ mtRatio + dtechGap + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()
# results6 = smf.ols('tenderRatio ~ mtRatio + 업체수TH + largeBusiness + db2014 + collude', data=df_final).fit()

# stargazer_tab = Stargazer([results1, results2, results3, results4, results5, results6])
# open('./V11_new/regression_v3-2(TH).html', 'w').write(stargazer_tab.render_html())  # for latex

# fitted = pd.concat([fit_final_col['tenderRatio'], results1.predict(fit_final_col), results2.predict(fit_final_col), results3.predict(fit_final_col),
#                     results4.predict(fit_final_col), results5.predict(fit_final_col), results6.predict(fit_final_col)], axis = 1)
# fitted.columns = ['tenderRatio','fitted1', 'fitted2', 'fitted3', 'fitted4', 'fitted5', 'fitted6']
# fitted_final = fitted.copy()
# fitted_final['diff1'] = fitted['tenderRatio'] - fitted['fitted1']
# fitted_final['diff2'] = fitted['tenderRatio'] - fitted['fitted2']
# fitted_final['diff3'] = fitted['tenderRatio'] - fitted['fitted3']
# fitted_final['diff4'] = fitted['tenderRatio'] - fitted['fitted4']
# fitted_final['diff5'] = fitted['tenderRatio'] - fitted['fitted5']
# fitted_final['diff6'] = fitted['tenderRatio'] - fitted['fitted6']
# fitted_final.to_csv('./V11_new/fitted_v3-2(TH).csv', index = False)
